In [1]:
import cv2
import numpy as np
import math
import torch

In [9]:
cap = cv2.VideoCapture('D:/AmirKabir/tez/ewap_dataset/ewap_dataset/seq_hotel/seq_hotel.avi')

while cap.isOpened():
    # Read a frame
    ret, frame = cap.read()

    # Check if the frame is valid
    if not ret:
        break

    # Display the frame
    cv2.imshow('Frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the video capture object
cap.release()
cv2.destroyAllWindows()

In [12]:
cap = cv2.VideoCapture('D:/AmirKabir/tez/ewap_dataset/ewap_dataset/seq_hotel/seq_hotel.avi')
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
total_frames

19350

In [31]:
# Open the video file
cap = cv2.VideoCapture('D:/AmirKabir/tez/ewap_dataset/ewap_dataset/seq_hotel/seq_hotel.avi')
# Set the frame position to 10
cap.set(cv2.CAP_PROP_POS_FRAMES, 19260)

# Read the frame at position 10
ret, frame = cap.read()

# Check if the frame is valid
if ret:
    # Display the frame
    cv2.imshow('Frame 11', frame)
    cv2.waitKey(0)
else:
    print("Error reading frame")

# Release the video capture object
cap.release()
cv2.destroyAllWindows()

In [1]:
import os
import math
import torch
import numpy as np
import cv2
from torch.utils.data import Dataset
from torch.utils.data.sampler import Sampler
from torch.utils.data.dataloader import DataLoader
from torchvision.transforms import transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def get_dataloader(data_dir, phase, obs_len, pred_len, batch_size):
    r"""Get dataloader for a specific phase

    Args:
        data_dir (str): path to the dataset directory
        phase (str): phase of the data, one of 'train', 'val', 'test'
        obs_len (int): length of observed trajectory
        pred_len (int): length of predicted trajectory
        batch_size (int): batch size

    Returns:
        loader_phase (torch.utils.data.DataLoader): dataloader for the specific phase
    """

    assert phase in ['train', 'val', 'test']
    
    
    data_set = data_dir + '/' + phase + '/'
    shuffle = True if phase == 'train' else False
    drop_last = True if phase == 'train' else False
    frame_path = data_dir 
    dataset_phase = TrajectoryDataset(data_set,frame_path = frame_path ,obs_len=obs_len, pred_len=pred_len)
    sampler_phase = None
    if batch_size > 1:
        sampler_phase = TrajBatchSampler(dataset_phase, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    loader_phase = DataLoader(dataset_phase, batch_sampler=sampler_phase ,collate_fn=traj_collate_fn, pin_memory=True)
    return loader_phase


def traj_collate_fn(data):
    r"""Collate function for the dataloader

    Args:
        data (list): list of tuples of (obs_seq, pred_seq, non_linear_ped, loss_mask, seq_start_end)

    Returns:
        obs_seq_list (torch.Tensor): (num_ped, obs_len, 2)
        pred_seq_list (torch.Tensor): (num_ped, pred_len, 2)
        non_linear_ped_list (torch.Tensor): (num_ped,)
        loss_mask_list (torch.Tensor): (num_ped, obs_len + pred_len)
        scene_mask (torch.Tensor): (num_ped, num_ped)
        seq_start_end (torch.Tensor): (num_ped, 2)
        frame_list_tensor
    """

    obs_seq_list, pred_seq_list, non_linear_ped_list, loss_mask_list, _,frame_list_tensor= zip(*data)

    _len = [len(seq) for seq in obs_seq_list]
    cum_start_idx = [0] + np.cumsum(_len).tolist()
    seq_start_end = [[start, end] for start, end in zip(cum_start_idx, cum_start_idx[1:])]
    seq_start_end = torch.LongTensor(seq_start_end)
    scene_mask = torch.zeros(sum(_len), sum(_len), dtype=torch.bool)
    for idx, (start, end) in enumerate(seq_start_end):
        scene_mask[start:end, start:end] = 1

    out = [torch.cat(obs_seq_list, dim=0), torch.cat(pred_seq_list, dim=0),
           torch.cat(non_linear_ped_list, dim=0), torch.cat(loss_mask_list, dim=0), scene_mask, seq_start_end,frame_list_tensor]
    return tuple(out)


class TrajBatchSampler(Sampler):
    r"""Samples batched elements by yielding a mini-batch of indices.
    Args:
        data_source (Dataset): dataset to sample from
        batch_size (int): Size of mini-batch.
        shuffle (bool, optional): set to ``True`` to have the data reshuffled
            at every epoch (default: ``False``).
        drop_last (bool): If ``True``, the sampler will drop the last batch if
            its size would be less than ``batch_size``
        generator (Generator): Generator used in sampling.
    """

    def __init__(self, data_source, batch_size=64, shuffle=False, drop_last=False, generator=None):
        self.data_source = data_source
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.drop_last = drop_last
        self.generator = generator

    def __iter__(self):
        assert len(self.data_source) == len(self.data_source.num_peds_in_seq)

        if self.shuffle:
            if self.generator is None:
                generator = torch.Generator()
                generator.manual_seed(int(torch.empty((), dtype=torch.int64).random_().item()))
            else:
                generator = self.generator
            indices = torch.randperm(len(self.data_source), generator=generator).tolist()
        else:
            indices = list(range(len(self.data_source)))
        num_peds_indices = self.data_source.num_peds_in_seq[indices]

        batch = []
        total_num_peds = 0
        for idx, num_peds in zip(indices, num_peds_indices):
            batch.append(idx)
            total_num_peds += num_peds
            if total_num_peds >= self.batch_size:
                yield batch
                batch = []
                total_num_peds = 0
        if len(batch) > 0 and not self.drop_last:
            yield batch

    def __len__(self):
        # Approximated number of batches.
        # The order of trajectories can be shuffled, so this number can vary from run to run.
        if self.drop_last:
            return sum(self.data_source.num_peds_in_seq) // self.batch_size
        else:
            return (sum(self.data_source.num_peds_in_seq) + self.batch_size - 1) // self.batch_size


def read_file(_path, delim='\t'):
    data = []
    if delim == 'tab':
        delim = '\t'
    elif delim == 'space':
        delim = ' '
    with open(_path, 'r') as f:
        for line in f:
            line = line.strip().split(delim)
            line = [float(i) for i in line]
            data.append(line)
    return np.asarray(data)


def poly_fit(traj, traj_len, threshold):
    """
    Input:
    - traj: Numpy array of shape (2, traj_len)
    - traj_len: Len of trajectory
    - threshold: Minimum error to be considered for non-linear traj
    Output:
    - int: 1 -> Non Linear 0-> Linear
    """
    t = np.linspace(0, traj_len - 1, traj_len)
    res_x = np.polyfit(t, traj[0, -traj_len:], 2, full=True)[1]
    res_y = np.polyfit(t, traj[1, -traj_len:], 2, full=True)[1]
    if res_x + res_y >= threshold:
        return 1.0
    else:
        return 0.0

def prepare_image(frame):
    transform = transforms.Compose([
                      transforms.ToPILImage(),
                      transforms.Resize(300),
                      transforms.ToTensor()])
    transformed_image = transform(frame)
    transformed_image = transformed_image.unsqueeze(0)
    return transformed_image.to(device)



# In your TrajectoryDataset class



class TrajectoryDataset(Dataset):
    """Dataloder for the Trajectory datasets"""

    def __init__(self, data_dir,frame_path ,obs_len=8, pred_len=12, skip=1, threshold=0.02, min_ped=1, delim='\t'):
        """
        Args:
        - data_dir: Directory containing dataset files in the format <frame_id> <ped_id> <x> <y>
        - obs_len: Number of time-steps in input trajectories
        - pred_len: Number of time-steps in output trajectories
        - skip: Number of frames to skip while making the dataset
        - threshold: Minimum error to be considered for non-linear traj when using a linear predictor
        - min_ped: Minimum number of pedestrians that should be in a sequence
        - delim: Delimiter in the dataset files
        """
        super(TrajectoryDataset, self).__init__()

        self.data_dir = data_dir
        self.obs_len = obs_len
        self.pred_len = pred_len
        self.skip = skip
        self.seq_len = self.obs_len + self.pred_len
        self.delim = delim
        self.frame_path = frame_path
        self.video_path = os.path.join(self.frame_path, 'seq_hotel.avi')

        all_files = os.listdir(self.data_dir)
        all_files = [os.path.join(self.data_dir, _path) for _path in all_files]
        num_peds_in_seq = []
        seq_list = []
        loss_mask_list = []
        non_linear_ped = []
        pedestrain_id = []#new add
        for path in all_files:
            
            try:
                data = read_file(path, delim)
            except:
                data = read_file(path, delim = 'space')
            frame_id = [] #new add    
            frames = np.unique(data[:, 0]).tolist()
            frame_data = []
            for frame in frames:
                frame_data.append(data[frame == data[:, 0], :])
            num_sequences = int(math.ceil((len(frames) - self.seq_len + 1) / skip))

            for idx in range(0, num_sequences * self.skip + 1, skip):
                curr_seq_data = np.concatenate(frame_data[idx:idx + self.seq_len], axis=0)
                frame_range = np.unique(curr_seq_data[:, 0])       #new add
                peds_in_curr_seq = np.unique(curr_seq_data[:, 1])
                curr_seq = np.zeros((len(peds_in_curr_seq), 2, self.seq_len))
                curr_loss_mask = np.zeros((len(peds_in_curr_seq), self.seq_len))
                num_peds_considered = 0
                _non_linear_ped = []
                ped_list = [] #new add
                for _, ped_id in enumerate(peds_in_curr_seq):
                    curr_ped_seq = curr_seq_data[curr_seq_data[:, 1] == ped_id, :]
                    curr_ped_seq = np.around(curr_ped_seq, decimals=4)
                    pad_front = frames.index(curr_ped_seq[0, 0]) - idx
                    pad_end = frames.index(curr_ped_seq[-1, 0]) - idx + 1
                    if pad_end - pad_front != self.seq_len:
                        continue
                    curr_ped_seq = np.transpose(curr_ped_seq[:, 2:])
                    curr_ped_seq = curr_ped_seq
                    _idx = num_peds_considered
                    curr_seq[_idx, :, pad_front:pad_end] = curr_ped_seq
                    # Linear vs Non-Linear Trajectory
                    _non_linear_ped.append(poly_fit(curr_ped_seq, pred_len, threshold))
                    curr_loss_mask[_idx, pad_front:pad_end] = 1
                    num_peds_considered += 1
                    ped_list.append(ped_id) #new add

                if num_peds_considered > min_ped:
                    non_linear_ped += _non_linear_ped
                    num_peds_in_seq.append(num_peds_considered)
                    loss_mask_list.append(curr_loss_mask[:num_peds_considered])
                    seq_list.append(curr_seq[:num_peds_considered])
                    frame_id.append(frame_range) #new add
                    pedestrain_id.append(ped_list) #new add
                    
            
            # cap = cv2.VideoCapture(self.frame_path + '/seq_hotel.avi')
            # frame_lists = [(row.min() , row.max())  for row in frame_id] #new add
            # images_list = []
            # for tup in frame_lists:
            #     temp_list = []
            #     current_frame_number = tup[0]
            #     for i in range(obs_len):
            #         cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame_number)
            #         ret, current_frame = cap.read()
            #         frame = prepare_image(current_frame).to(device)
            #         temp_list.append(frame)
            #         current_frame_number+=10
            #     images_list.append(temp_list)
            # import pickle

            # Specify the full path to save the images_list as a pickle file
            # save_path = 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/image_List_hotel/images_list.pkl'
            # with open(save_path, 'wb') as f:
            #     pickle.dump(images_list, f)
            
        # with open(save_path, 'rb') as f:
        self.frame_lists = [(row.min(), row.max()) for row in frame_id]
        # self.images = extract_frames(video_path, frame_lists, self.obs_len)
        self.num_seq = len(seq_list)
        seq_list = np.concatenate(seq_list, axis=0)
        loss_mask_list = np.concatenate(loss_mask_list, axis=0)
        non_linear_ped = np.asarray(non_linear_ped)
         #new add
        self.pedestrain_id = np.concatenate(pedestrain_id , axis = 0) #new add
        self.num_peds_in_seq = np.array(num_peds_in_seq) 

        # Convert numpy -> Torch Tensor
        self.obs_traj = torch.from_numpy(seq_list[:, :, :self.obs_len]).type(torch.float).permute(0, 2, 1)  # NTC
        self.pred_traj = torch.from_numpy(seq_list[:, :, self.obs_len:]).type(torch.float).permute(0, 2, 1)  # NTC
        self.loss_mask = torch.from_numpy(loss_mask_list).type(torch.float)
        self.non_linear_ped = torch.from_numpy(non_linear_ped).type(torch.float)
        cum_start_idx = [0] + np.cumsum(num_peds_in_seq).tolist()
        self.seq_start_end = [(start, end) for start, end in zip(cum_start_idx, cum_start_idx[1:])]

    def __len__(self):
        return self.num_seq
    
    def extract_frames(self, video_path, min_frame, max_frame, obs_len):
        images_list = []
        cap =  cv2.VideoCapture(video_path) 
        temp_list = []
        current_frame_number = min_frame
        for _ in range(obs_len):
            cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame_number)
            ret, current_frame = cap.read()
            if not ret:
                raise RuntimeError(f"Could not read frame {current_frame_number} from {video_path}")
            frame = prepare_image(current_frame).to(device)
            temp_list.append(frame)
            current_frame_number += 10
        images_list.append(temp_list)
        return images_list

    def __getitem__(self, index):
        start, end = self.seq_start_end[index]
        min_frame, max_frame= self.frame_lists[index]
        frames = self.extract_frames(self.video_path, min_frame, max_frame, self.obs_len)
        out = [self.obs_traj[start:end], self.pred_traj[start:end],
               self.non_linear_ped[start:end], self.loss_mask[start:end], [[0, end - start]], frames]
        return out 


In [2]:
dataset_dir = 'D:/AmirKabir/tez/eighen trajectory/project_git/project/project_test_cpu/datasets/test'
loader_train = get_dataloader(dataset_dir, 'train', 8, 12 ,batch_size=1)

In [3]:
all_data = []
for data  in loader_train:
    all_data.append(data)

In [ ]:
len(all_data)

In [3]:
for batch in loader_train:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            obs_traj, pred_traj = [tensor.to(device, non_blocking=True) for tensor in batch[:2]]
            frames = [[tensor.to(device, non_blocking=True) for tensor in sublist] for sublist in batch[-1][0]]

In [10]:
frames[0]

[tensor([[[[0.4980, 0.4118, 0.4588,  ..., 0.5490, 0.5020, 0.5686],
           [0.3529, 0.3020, 0.3255,  ..., 0.5608, 0.5020, 0.6078],
           [0.2667, 0.2863, 0.4353,  ..., 0.5412, 0.5176, 0.6039],
           ...,
           [0.2784, 0.2784, 0.2784,  ..., 0.6902, 0.6745, 0.6627],
           [0.2627, 0.2627, 0.2627,  ..., 0.6980, 0.6824, 0.6784],
           [0.2549, 0.2549, 0.2549,  ..., 0.7098, 0.6980, 0.6902]],
 
          [[0.5059, 0.4196, 0.4667,  ..., 0.5647, 0.5176, 0.5843],
           [0.3608, 0.3098, 0.3333,  ..., 0.5765, 0.5176, 0.6235],
           [0.2745, 0.2941, 0.4431,  ..., 0.5569, 0.5333, 0.6196],
           ...,
           [0.2980, 0.2980, 0.2980,  ..., 0.6784, 0.6627, 0.6510],
           [0.2824, 0.2824, 0.2824,  ..., 0.6863, 0.6706, 0.6667],
           [0.2745, 0.2745, 0.2745,  ..., 0.6980, 0.6863, 0.6784]],
 
          [[0.5333, 0.4471, 0.4941,  ..., 0.6000, 0.5529, 0.6196],
           [0.3882, 0.3373, 0.3608,  ..., 0.6118, 0.5529, 0.6588],
           [0.3020, 0.32

In [8]:
len(frames[0])

8

In [2]:
def read_file(_path, delim='\t'):
    data = []
    if delim == 'tab':
        delim = '\t'
    elif delim == 'space':
        delim = ' '
    with open(_path, 'r') as f:
        for line in f:
            line = line.strip().split(delim)
            line = [float(i) for i in line]
            data.append(line)
    return np.asarray(data)

In [3]:
def poly_fit(traj, traj_len, threshold):
    """
    Input:
    - traj: Numpy array of shape (2, traj_len)
    - traj_len: Len of trajectory
    - threshold: Minimum error to be considered for non-linear traj
    Output:
    - int: 1 -> Non Linear 0-> Linear
    """
    t = np.linspace(0, traj_len - 1, traj_len)
    res_x = np.polyfit(t, traj[0, -traj_len:], 2, full=True)[1]
    res_y = np.polyfit(t, traj[1, -traj_len:], 2, full=True)[1]
    if res_x + res_y >= threshold:
        return 1.0
    else:
        return 0.0

In [4]:
num_peds_in_seq = []
seq_list = []
loss_mask_list = []
non_linear_ped = []
frame_id = []
path = 'D:/AmirKabir/tez/eighen trajectory/nuscenes_test_cpu/nuscenes_test_cpu/datasets/eth/train/biwi_hotel_train.txt'
try:
    data = read_file(path)
except:
    data = read_file(path, delim = 'space')
        
frames = np.unique(data[:, 0]).tolist()
frame_data = []
for frame in frames:
    frame_data.append(data[frame == data[:, 0], :])

In [35]:
seq_len = 20
obs_len = 8
skip = 1
pred_len = 12
threshold=0.02
min_ped=1
num_sequences = int(math.ceil((len(frames) - seq_len + 1) / skip))
num_peds_in_seq = []
seq_list = []
loss_mask_list = []
non_linear_ped = []
frame_id = []
pedestrain_id = []
for idx in range(0, num_sequences * skip + 1, skip):
    curr_seq_data = np.concatenate(frame_data[idx:idx + seq_len], axis=0)
    peds_in_curr_seq = np.unique(curr_seq_data[:, 1])
    frame_range = np.unique(curr_seq_data[:, 0])
    curr_seq = np.zeros((len(peds_in_curr_seq), 2, seq_len))
    curr_loss_mask = np.zeros((len(peds_in_curr_seq), seq_len))
    num_peds_considered = 0
    _non_linear_ped = []
    ped_list = []
    for _, ped_id in enumerate(peds_in_curr_seq):
        curr_ped_seq = curr_seq_data[curr_seq_data[:, 1] == ped_id, :]
        curr_ped_seq = np.around(curr_ped_seq, decimals=4)
        pad_front = frames.index(curr_ped_seq[0, 0]) - idx
        pad_end = frames.index(curr_ped_seq[-1, 0]) - idx + 1
        if pad_end - pad_front != seq_len:
            continue
        curr_ped_seq = np.transpose(curr_ped_seq[:, 2:])
        curr_ped_seq = curr_ped_seq
        _idx = num_peds_considered
        curr_seq[_idx, :, pad_front:pad_end] = curr_ped_seq
        # Linear vs Non-Linear Trajectory
        _non_linear_ped.append(poly_fit(curr_ped_seq, pred_len, threshold))
        curr_loss_mask[_idx, pad_front:pad_end] = 1
        num_peds_considered += 1
        ped_list.append(ped_id)


    if num_peds_considered > min_ped:
        non_linear_ped += _non_linear_ped
        num_peds_in_seq.append(num_peds_considered)
        loss_mask_list.append(curr_loss_mask[:num_peds_considered])
        seq_list.append(curr_seq[:num_peds_considered])
        frame_id.append(frame_range)
        pedestrain_id.append(ped_list)

In [36]:
num_seq = len(seq_list)
seq_list = np.concatenate(seq_list, axis=0)
loss_mask_list = np.concatenate(loss_mask_list, axis=0)
frame_lists = [(row.min() , row.max())  for row in frame_id]
non_linear_ped = np.asarray(non_linear_ped)
num_peds_in_seq = np.array(num_peds_in_seq)
pedestrain_id = np.concatenate(pedestrain_id , axis = 0)
# Convert numpy -> Torch T
obs_traj = torch.from_numpy(seq_list[:, :, :obs_len]).type(torch.float).permute(0, 2, 1)  # NTC
pred_traj = torch.from_numpy(seq_list[:, :, obs_len:]).type(torch.float).permute(0, 2, 1)  # NTC
loss_mask = torch.from_numpy(loss_mask_list).type(torch.float)
non_linear_ped = torch.from_numpy(non_linear_ped).type(torch.float)
cum_start_idx = [0] + np.cumsum(num_peds_in_seq).tolist()
seq_start_end = [(start, end) for start, end in zip(cum_start_idx, cum_start_idx[1:])]

In [45]:
num_peds_in_seq

array([3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
       3, 3, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5,
       5, 5, 5, 3, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 3, 3, 3, 3, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 3, 4, 5, 2, 2, 2, 3, 2, 2, 2, 5, 5, 5, 3, 3,
       5, 5, 5, 4, 5, 3, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 4, 3, 3,
       2, 2, 2, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 2,
       2, 2, 2, 2, 4, 4, 4, 4, 4, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 6, 6, 6, 6, 8, 7, 5, 5, 5])

In [15]:
obs_traj_test.shape

torch.Size([3, 8, 2])

In [23]:
frame_lists


[(0.0, 190.0),
 (10.0, 200.0),
 (20.0, 210.0),
 (30.0, 220.0),
 (40.0, 230.0),
 (50.0, 240.0),
 (60.0, 250.0),
 (500.0, 690.0),
 (510.0, 700.0),
 (520.0, 710.0),
 (530.0, 720.0),
 (540.0, 730.0),
 (550.0, 740.0),
 (560.0, 750.0),
 (570.0, 760.0),
 (580.0, 770.0),
 (590.0, 780.0),
 (600.0, 790.0),
 (610.0, 800.0),
 (1150.0, 1340.0),
 (1160.0, 1350.0),
 (1170.0, 1360.0),
 (1180.0, 1370.0),
 (1190.0, 1380.0),
 (2770.0, 2960.0),
 (2780.0, 2970.0),
 (2790.0, 2980.0),
 (4000.0, 4190.0),
 (4010.0, 4200.0),
 (4020.0, 4210.0),
 (4030.0, 4220.0),
 (4040.0, 4230.0),
 (4050.0, 4240.0),
 (4060.0, 4250.0),
 (4070.0, 4260.0),
 (4430.0, 4620.0),
 (4440.0, 4630.0),
 (4450.0, 4640.0),
 (4460.0, 4650.0),
 (4470.0, 4660.0),
 (4480.0, 4670.0),
 (4490.0, 4680.0),
 (4500.0, 4690.0),
 (4510.0, 4700.0),
 (4520.0, 4710.0),
 (4530.0, 4720.0),
 (4540.0, 4730.0),
 (4550.0, 4740.0),
 (4560.0, 4750.0),
 (4570.0, 4760.0),
 (4580.0, 4770.0),
 (4590.0, 4780.0),
 (4600.0, 4790.0),
 (4610.0, 4800.0),
 (4620.0, 4810.0),
 

In [121]:
lis

[(0, 190),
 (10, 200),
 (20, 210),
 (30, 220),
 (40, 230),
 (50, 240),
 (60, 250),
 (500, 690),
 (510, 700),
 (520, 710),
 (530, 720),
 (540, 730),
 (550, 740),
 (560, 750),
 (570, 760),
 (580, 770),
 (590, 780),
 (600, 790),
 (610, 800),
 (1150, 1340),
 (1160, 1350),
 (1170, 1360),
 (1180, 1370),
 (1190, 1380),
 (2770, 2960),
 (2780, 2970),
 (2790, 2980),
 (4000, 4190),
 (4010, 4200),
 (4020, 4210),
 (4030, 4220),
 (4040, 4230),
 (4050, 4240),
 (4060, 4250),
 (4070, 4260),
 (4430, 4620),
 (4440, 4630),
 (4450, 4640),
 (4460, 4650),
 (4470, 4660),
 (4480, 4670),
 (4490, 4680),
 (4500, 4690),
 (4510, 4700),
 (4520, 4710),
 (4530, 4720),
 (4540, 4730),
 (4550, 4740),
 (4560, 4750),
 (4570, 4760),
 (4580, 4770),
 (4590, 4780),
 (4600, 4790),
 (4610, 4800),
 (4620, 4810),
 (4630, 4820),
 (4640, 4830),
 (4650, 4840),
 (4660, 4850),
 (4670, 4860),
 (4680, 4870),
 (4690, 4880),
 (4700, 4890),
 (4710, 4900),
 (4720, 4910),
 (4730, 4920),
 (4740, 4930),
 (4750, 4940),
 (4760, 4950),
 (4770, 496

In [58]:
seq_list[1]

array([[-1.72, -1.72, -1.72, -1.72, -1.72, -1.72, -1.72, -1.72, -1.72,
        -1.72, -1.72, -1.7 , -1.7 , -1.7 , -1.7 , -1.7 , -1.7 , -1.7 ,
        -1.7 , -1.7 ],
       [ 1.32,  1.32,  1.32,  1.32,  1.32,  1.32,  1.32,  1.32,  1.32,
         1.32,  1.32,  1.32,  1.32,  1.32,  1.32,  1.32,  1.32,  1.32,
         1.32,  1.32]])

In [19]:
for data in loader_train:
    data

In [25]:
data[1]

tensor([[[ 8.0368,  9.7595],
         [ 8.5590,  9.5941],
         [ 9.0889,  9.4349],
         [ 9.6191,  9.2653],
         [10.1490,  9.0531],
         [10.6792,  8.8409],
         [11.2456,  8.6915],
         [11.8481,  8.6049],
         [12.4444,  8.4753],
         [13.0253,  8.2462],
         [13.6061,  8.0171],
         [14.1433,  7.8023]],

        [[ 8.0023,  9.1593],
         [ 8.4544,  8.9648],
         [ 8.9692,  8.7426],
         [ 9.4996,  8.5135],
         [10.0386,  8.2913],
         [10.5973,  8.0856],
         [11.1561,  7.8798],
         [11.7370,  7.6770],
         [12.3507,  7.4779],
         [12.9647,  7.2791],
         [13.5407,  7.1094],
         [14.1167,  6.9400]]])